In [ ]:
kaggle=1
collab=0

In [ ]:
if kaggle:
    import os
    %cd /kaggle/input/nepsense/Model Training
    print(os.getcwd())

In [ ]:
if collab:
  from google.colab import drive
  drive.mount('/content/drive')
#prashantpath
  %cd /content/drive/MyDrive/Projects/NepSense/Prashant/Model Training

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# from gensim.models import Word2Vec
# from keras_preprocessing import pad_sequences


In [ ]:
preprocessed_filename="preprocessed_data.xlsx"
dataset=f"../Preprocessing/Preprocessing Datas/Outputs/{preprocessed_filename}"
# Load Excel file into a DataFrame
df = pd.read_excel(dataset)
df.head()

In [ ]:
# df.head()
# Find NaN values in the preprocessing_text column
print(df['preprocessing_text'].isna().sum())
# Remove rows with NaN values in the preprocessing_text column
df = df.dropna(subset=['preprocessing_text'])
df["Profanity"].fillna(0)
df["Offensive"].fillna(0)

In [ ]:
df=df[(df["Gender"] == 1) | (df["Gender"]==0)]

In [ ]:
def label_assignment(row):
    if row['Profanity'] == 1 and row['Offensive'] == 1:
        return 2
    elif row['Profanity'] == 0 and row['Offensive'] == 1:
        return 1
    else:
        return 0

df["Label"]=df.apply(label_assignment, axis=1)
df.head()

In [ ]:
sns.countplot(x="Label",data=df)
plt.show()

In [ ]:
def get_genderplot(df):
    profanity_df=df[df["Profanity"]==1]
    offensive_df=df[df["Offensive"]==1]

    # Create subplots
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    # Count plot for 'Gender' column in 'profanity_df'
    sns.countplot(x='Gender', data=profanity_df, ax=axs[0])
    axs[0].set_title('Count Plot for Gender (Profanity = 1)')
    axs[0].set_xlabel('Gender')
    axs[0].set_ylabel('Count')

    # Count plot for 'Gender' column in 'offensive_df'
    sns.countplot(x='Gender', data=offensive_df, ax=axs[1])
    axs[1].set_title('Count Plot for Gender (Offensive = 1)')
    axs[1].set_xlabel('Gender')
    axs[1].set_ylabel('Count')

    plt.tight_layout()
    plt.show()

get_genderplot(df)


In [ ]:
get_genderplot(df)

In [ ]:
df.head()

In [ ]:
df['Combined_Label'] = df.apply(lambda row: [row['Gender'], row['Label']], axis=1)

# Display the distribution of the combined labels
print("Original combined label distribution:")
print(df['Combined_Label'].value_counts())


In [ ]:
def balance(df,column):
  count=df[column].value_counts().min()
  df=df.groupby(column).apply(lambda x:x.sample(count)).reset_index(drop=True)
  return df

df=balance(df,"Label")
sns.countplot(x='Label', data=df)

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize


df["tokenized_text"]=df["preprocessing_text"].apply(lambda text: word_tokenize(text))
df.head()


In [ ]:
import torch
from transformers import BertModel, BertTokenizer
from nltk.util import ngrams
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm  # Import tqdm for progress tracking

# Check if CUDA is available
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load multilingual BERT model and tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
multi_output_multi_label_model = BertModel.from_pretrained(model_name)

In [ ]:
# Move the model to the GPU if available
multi_output_multi_label_model.to(device)

# Set the model to evaluation mode
multi_output_multi_label_model.eval()

def get_ngram_embeddings(text, n):
    # Tokenize the input text
    tokenized_text = tokenizer.encode(text, add_special_tokens=True)

    # Generate n-grams from tokenized text
    text_ngrams = list(ngrams(tokenized_text, n))

    # Initialize list to store embeddings
    embeddings = []

    # Iterate through n-grams and get embeddings with tqdm for progress tracking
    for gram in tqdm(text_ngrams, desc="Generating Embeddings", unit="n-gram"):
        # Convert n-gram to torch tensor and move it to the GPU
        input_ids = torch.tensor(gram).unsqueeze(0).to(device)  # Add batch dimension and move to device

        # Get the embeddings from the BERT model
        with torch.no_grad():
            outputs = multi_output_multi_label_model(input_ids)

        # Extract the output embeddings from the last layer
        last_hidden_states = outputs.last_hidden_state

        # Average pooling to get sentence-level embedding
        sentence_embedding = torch.mean(last_hidden_states, dim=1).squeeze().cpu().numpy()  # Move to CPU for numpy conversion

        # Append the embedding to the list
        embeddings.append(sentence_embedding)

    # Convert list of embeddings to numpy array
    embeddings_array = np.array(embeddings)

    return embeddings_array

def embed_text_with_ngrams(df, n):
    # Create a new column to store embeddings
    df['embeddings'] = df['preprocessing_text'].apply(lambda text: get_ngram_embeddings(text, n))

    return df

# Example usage:
# Assuming 'df' is your DataFrame and 'preprocessing_text' is the column containing preprocessed text
# 'n' is the value of n for n-grams
df_with_embeddings = embed_text_with_ngrams(df, 2)
# Determine the maximum sequence length
max_len = max(df['embeddings'].apply(len))
# Pad sequences so that all embeddings have the same length
padded_embeddings = pad_sequences(df['embeddings'].values, maxlen=max_len, padding='post', dtype='float32')

In [ ]:
output_dir = '/kaggle/working/embedding_added.csv'
df.to_csv(output_dir)
print(f"Embedding added File saved successfully (path: {output_dir})")

In [ ]:
output_dir = '/kaggle/working/embedding_added.xlsx'
df.to_excel(output_dir)
print(f"Embedding added File saved successfully (path: {output_dir})")

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
encoder = OneHotEncoder(sparse=False)
gender = df["Gender"].to_numpy()
profanity = df["Label"].to_numpy()

# Initialize label encoders
gender_encoder = LabelEncoder()
profanity_encoder = LabelEncoder()

# Encode gender and profanity labels
y_gender = gender_encoder.fit_transform(gender)
y_profanity = profanity_encoder.fit_transform(profanity)

# For binary classification, no need to perform one-hot encoding
# For multi-class classification, perform one-hot encoding

y_profanity = to_categorical(y_profanity)
print(y_profanity)


In [ ]:
# Split the data
X_train, X_test, y_gender_train, y_gender_test, y_profanity_train, y_profanity_test = train_test_split(padded_embeddings, y_gender, y_profanity, test_size=0.2, random_state=42)

In [ ]:
from keras.constraints import max_norm
import tensorflow
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dropout, Bidirectional, LSTM, Dense
import tensorflow as tf
regularise = tensorflow.keras.regularizers.l2(0.0001)

# Define the input layer with the shape of your embedded input
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))

# Dropout layer
dropout_layer = Dropout(0.4)(input_layer)

# First bidirectional LSTM layer
lstm_1 = Bidirectional(LSTM(512, return_sequences=True))(dropout_layer)

lstm_1 = Bidirectional(LSTM(256, return_sequences=True))(lstm_1)
# Second bidirectional LSTM layer
lstm_2 = Bidirectional(LSTM(32, kernel_constraint=max_norm(3)))(lstm_1)

# Intermediate dense layer
dense_layer = Dense(32, activation='relu', kernel_regularizer=regularise)(lstm_2)

# Output layers
gender_output = Dense(1, activation='sigmoid', name='gender_output')(dense_layer)
profanity_output = Dense(3, activation='softmax', name='profanity_output')(dense_layer)

# Define the model
multi_output_multi_label_model = Model(inputs=input_layer, outputs=[gender_output, profanity_output])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# Compile the model
multi_output_multi_label_model.compile(
    optimizer=optimizer,
    loss={
        'gender_output': 'binary_crossentropy',
        'profanity_output': 'categorical_crossentropy'
    },
    metrics={
        'gender_output': 'accuracy',
        'profanity_output': 'accuracy'
    }
)


print(multi_output_multi_label_model.summary())
from keras.callbacks import EarlyStopping
import os
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = multi_output_multi_label_model.fit(
    X_train,  # Training input data
    {'gender_output': y_gender_train, 'profanity_output': y_profanity_train},  # Training target data
    epochs=100,  # Number of epochs
    batch_size=128,  # Batch size
    validation_split=0.2,  # Percentage of training data to use as validation data
    callbacks=[early_stopping]  # Early stopping callback
)
# Plot training history
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.plot(history.history['profanity_output_accuracy'], label='profanity_output_accuracy')
plt.plot(history.history['val_profanity_output_accuracy'], label='val_profanity_output_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Profanity Accuracy')
plt.legend()
plt.show()
plt.plot(history.history['gender_output_accuracy'], label='gender_output_accuracy')
plt.plot(history.history['val_gender_output_accuracy'], label='val_gender_output_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Gender Accuracy')
plt.legend()
plt.show()




import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Step 1: Make predictions
predictions = multi_output_multi_label_model.predict(X_test)
gender_pred = predictions[0]
profanity_pred = predictions[1]

# Step 2: Convert predictions to binary and categorical labels
gender_pred_labels = (gender_pred > 0.5).astype(int).flatten()
profanity_pred_labels = np.argmax(profanity_pred, axis=1)

# Assuming y_test_gender is binary and y_test_profanity is categorical
y_test_gender_labels = y_gender_test.flatten()
y_test_profanity_labels = np.argmax(y_profanity_test, axis=1)

# Step 3: Create confusion matrices
conf_matrix_gender = confusion_matrix(y_test_gender_labels, gender_pred_labels)
conf_matrix_profanity = confusion_matrix(y_test_profanity_labels, profanity_pred_labels)

# Step 4: Plot confusion matrices
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Gender confusion matrix
disp_gender = ConfusionMatrixDisplay(confusion_matrix=conf_matrix_gender, display_labels=[0, 1])
disp_gender.plot(ax=ax[0], cmap=plt.cm.Blues)
ax[0].set_title('Gender Confusion Matrix')

# Profanity confusion matrix
disp_profanity = ConfusionMatrixDisplay(confusion_matrix=conf_matrix_profanity, display_labels=[0, 1, 2])
disp_profanity.plot(ax=ax[1], cmap=plt.cm.Blues)
ax[1].set_title('Profanity Confusion Matrix')

plt.show()


from sklearn.metrics import classification_report

# Assuming y_test_gender is binary and y_test_profanity is categorical
y_test_gender_labels = y_gender_test.flatten()
y_test_profanity_labels = np.argmax(y_profanity_test, axis=1)

# Gender classification report
gender_report = classification_report(y_test_gender_labels, gender_pred_labels, target_names=['Not Gender', 'Gender'])
print("Gender Classification Report:")
print(gender_report)

# Profanity classification report
profanity_report = classification_report(y_test_profanity_labels, profanity_pred_labels, target_names=['Class 0', 'Class 1', 'Class 2'])
print("Profanity Classification Report:")
print(profanity_report)

In [ ]:
plot_model(multi_output_multi_label_model, to_file='/kaggle/working/multi_output_multi_label_model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Save the model architecture and weights
multi_output_multi_label_model.save('/kaggle/working/multi_output_multi_label_multi_output.h5')

In [ ]:
import pickle

# Save the tokenizer
with open('/kaggle/working/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


In [ ]:
from keras.constraints import max_norm
import tensorflow
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dropout, Bidirectional, LSTM, Dense

regularise = tensorflow.keras.regularizers.l2(0.0001)

# Define the input layer with the shape of your embedded input
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))

# Dropout layer
dropout_layer = Dropout(0.4)(input_layer)

# First bidirectional LSTM layer
lstm_1 = Bidirectional(LSTM(512, return_sequences=True))(dropout_layer)

lstm_1 = Bidirectional(LSTM(256, return_sequences=True))(lstm_1)
# Second bidirectional LSTM layer
lstm_2 = Bidirectional(LSTM(32, kernel_constraint=max_norm(3)))(lstm_1)

# Intermediate dense layer
dense_layer = Dense(32, activation='relu', kernel_regularizer=regularise)(lstm_2)

# Output layers
gender_output = Dense(1, activation='sigmoid', name='gender_output')(dense_layer)
profanity_output = Dense(3, activation='softmax', name='profanity_output')(dense_layer)

# Define the model
multi_output_multi_label_model = Model(inputs=input_layer, outputs=[gender_output, profanity_output])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
# Compile the model
multi_output_multi_label_model.compile(
    optimizer=optimizer,
    loss={
        'gender_output': 'binary_crossentropy',
        'profanity_output': 'categorical_crossentropy'
    },
    metrics={
        'gender_output': 'accuracy',
        'profanity_output': 'accuracy'
    }
)


print(multi_output_multi_label_model.summary())
from keras.callbacks import EarlyStopping
import os
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = multi_output_multi_label_model.fit(
    X_train,  # Training input data
    {'gender_output': y_gender_train, 'profanity_output': y_profanity_train},  # Training target data
    epochs=100,  # Number of epochs
    batch_size=128,  # Batch size
    validation_split=0.2,  # Percentage of training data to use as validation data
    callbacks=[early_stopping]  # Early stopping callback
)
# Plot training history
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.plot(history.history['profanity_output_accuracy'], label='profanity_output_accuracy')
plt.plot(history.history['val_profanity_output_accuracy'], label='val_profanity_output_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Profanity Accuracy')
plt.legend()
plt.show()
plt.plot(history.history['gender_output_accuracy'], label='gender_output_accuracy')
plt.plot(history.history['val_gender_output_accuracy'], label='val_gender_output_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Gender Accuracy')
plt.legend()
plt.show()




import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Step 1: Make predictions
predictions = multi_output_multi_label_model.predict(X_test)
gender_pred = predictions[0]
profanity_pred = predictions[1]

# Step 2: Convert predictions to binary and categorical labels
gender_pred_labels = (gender_pred > 0.5).astype(int).flatten()
profanity_pred_labels = np.argmax(profanity_pred, axis=1)

# Assuming y_test_gender is binary and y_test_profanity is categorical
y_test_gender_labels = y_gender_test.flatten()
y_test_profanity_labels = np.argmax(y_profanity_test, axis=1)

# Step 3: Create confusion matrices
conf_matrix_gender = confusion_matrix(y_test_gender_labels, gender_pred_labels)
conf_matrix_profanity = confusion_matrix(y_test_profanity_labels, profanity_pred_labels)

# Step 4: Plot confusion matrices
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Gender confusion matrix
disp_gender = ConfusionMatrixDisplay(confusion_matrix=conf_matrix_gender, display_labels=[0, 1])
disp_gender.plot(ax=ax[0], cmap=plt.cm.Blues)
ax[0].set_title('Gender Confusion Matrix')

# Profanity confusion matrix
disp_profanity = ConfusionMatrixDisplay(confusion_matrix=conf_matrix_profanity, display_labels=[0, 1, 2])
disp_profanity.plot(ax=ax[1], cmap=plt.cm.Blues)
ax[1].set_title('Profanity Confusion Matrix')

plt.show()


from sklearn.metrics import classification_report

# Assuming y_test_gender is binary and y_test_profanity is categorical
y_test_gender_labels = y_gender_test.flatten()
y_test_profanity_labels = np.argmax(y_profanity_test, axis=1)

# Gender classification report
gender_report = classification_report(y_test_gender_labels, gender_pred_labels, target_names=['Not Gender', 'Gender'])
print("Gender Classification Report:")
print(gender_report)

# Profanity classification report
profanity_report = classification_report(y_test_profanity_labels, profanity_pred_labels, target_names=['Class 0', 'Class 1', 'Class 2'])
print("Profanity Classification Report:")
print(profanity_report)